In [1]:
import os
# API_GROQ = "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"

In [244]:
from typing import Iterable, Any
from pdfminer.high_level import extract_pages, extract_text
import os
from PyPDF2 import PdfReader
from groq import Groq

def perguntaGroq(textoExtraido):
    clientGroq = Groq(api_key="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")
    
    prompt = f"""
    Incorpore um especialista em verificação de Projetos. Com base no texto fornecido, identifique se estão presentes as seguintes informações:

    1. Título do Projeto: Identifique se há um título que possa ser reconhecido como o nome do projeto.
    2. Categoria Disputada: Determine se há menção a uma categoria como curso técnico, graduação, pós-graduação, ou outras especificadas posteriormente.
    3. Nome da Equipe: Verifique se há um nome que identifica uma equipe.
    4. Nomes dos Integrantes ou Nome do Integrante: Identifique se são listados os nomes dos integrantes ou de um integrante específico.
    5. Cursos que Estão Matriculados: Verifique se há menção aos cursos nos quais os integrantes estão matriculados.

    Instruções adicionais para o modelo:

    - Se um campo não estiver claramente identificado, informe que o campo não foi encontrado.
    - Responda em um formato estruturado, listando cada item como um tópico (checklist) seguido de “Encontrado” ou “Não Encontrado”. Inclua o trecho do texto correspondente, se possível.
    - Utilize ícones de check (✔️) e uncheck (❌) para indicar a presença ou ausência de cada informação.  
    - Responda apenas o checklist solicitado
    - Responda em pt-br

    Texto extraído:
    {textoExtraido}
    """

    chat_completion = clientGroq.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )
    
    return chat_completion.choices[0].message.content



def show_ltitem_hierarchy(o: Any, depth=0, page_num=0, errors=None):
    """Show location and text of LTItem and all its descendants"""
    if errors is None:
        errors = []

    '''
    if depth == 0:
        print('element                        fontname             text')
        print('------------------------------ -------------------- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '
        f'{get_optional_fontinfo(o):<20.20s} '
        f'{get_optional_text(o)}'
    )
    '''

    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        # Verifica se a fonte é Arial e se o tamanho é pelo menos 10
        if 'Arial' not in o.fontname or o.size < 9.:
            errors.append({
                'page': page_num,
                'text': get_optional_text(o),
                'fontname': o.fontname,
                'size': o.size,
                'error': 'Fonte diferente de Arial' if 'Arial' not in o.fontname else 'Tamanho inferior a 10'
            })

    if isinstance(o, Iterable):
        for i in o:
            show_ltitem_hierarchy(i, depth=depth + 1, page_num=page_num, errors=errors)

    return errors

def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of class"""
    return '  ' * depth + o.__class__.__name__

def get_optional_fontinfo(o: Any) -> str:
    """Font info of LTChar if available, otherwise empty string"""
    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        return f'{o.fontname} {round(o.size)}pt'
    return ''

def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):
        return o.get_text().strip()
    return ''


def verificar_formato_a4(pdf_reader):
    # Dimensões de uma página A4 em pontos com margem de 10%
    largura_a4 = 595.28
    altura_a4 = 841.89

    #margem = 0.001  # 0.1% de margem - Nao preicsa de tanto
    margem = 0.01  # 1% de margem

    largura_min = largura_a4 * (1 - margem)
    largura_max = largura_a4 * (1 + margem)
    altura_min = altura_a4 * (1 - margem)
    altura_max = altura_a4 * (1 + margem)

    for pagina in pdf_reader.pages:
        largura = float(pagina.mediabox[2])
        altura = float(pagina.mediabox[3])
        
        # Verifica se as dimensões estão dentro da margem de 10%
        if not (largura_min <= largura <= largura_max and altura_min <= altura <= altura_max):
            return False, largura, altura
    return True, None, None


def extrair_texto_primeira_pagina(caminho_arquivo):
    # Verifica se o arquivo existe
    if not os.path.isfile(caminho_arquivo):
        return f"Arquivo '{caminho_arquivo}' não encontrado ❌."

    # Verifica se o arquivo é um PDF
    if not caminho_arquivo.lower().endswith('.pdf'):
        return "O arquivo não é um PDF ❌."

    try:
        # Extrai o texto da primeira página
        texto_primeira_pagina = extract_text(caminho_arquivo, page_numbers=[0])
        
        if texto_primeira_pagina:
            #print("Texto extraído da primeira página:\n")
            CheckListCapa = perguntaGroq (texto_primeira_pagina)
            print ("Check List Capa:\n")
            print (CheckListCapa)

        else:
            print("Não foi possível extrair o texto da primeira página ou a página está vazia ❌.")

    except Exception as e:
        return f"Erro ao tentar extrair o texto da primeira página: {e}"
    




def verificar_pdf(caminho_arquivo):
    # Verifica se o arquivo existe
    if not os.path.isfile(caminho_arquivo):
        return f"Arquivo '{caminho_arquivo}' não encontrado ❌."

    # Verifica se o arquivo é um PDF
    if not caminho_arquivo.lower().endswith('.pdf'):
        return "O arquivo não é um PDF ❌."

    try:
        # Abre e lê o PDF
        with open(caminho_arquivo, 'rb') as arquivo_pdf:
            leitor_pdf = PdfReader(arquivo_pdf)
            numero_paginas = len(leitor_pdf.pages)
            
            if numero_paginas < 22:
                paginas_status = f"O arquivo é um PDF ✔️.\nO Arquivo tem menos de 22 páginas ✔️. Total de páginas: {numero_paginas}."
            else:
                paginas_status = f"O arquivo é um PDF ✔️.\nO Arquivo tem 22 páginas ou mais ❌. Total de páginas: {numero_paginas}."
    
    except Exception as e:
        return f"Erro ao tentar ler o PDF: {e}"

    try:

        # Verifica se o documento está no formato A4
        is_a4, largura, altura = verificar_formato_a4(leitor_pdf)
        if not is_a4:
            return f"O documento não está no formato A4 ❌. Formato encontrado: {largura} x {altura} pontos."
        else:
            print("O documento está no formato A4 ✔️.")
        

        # Extrai e exibe o texto da primeira página
        texto_primeira_pagina = extrair_texto_primeira_pagina(caminho_arquivo)
        #CheckListCapa = perguntaGroq (texto_primeira_pagina)
        #print ("Check List Capa:\n")
        #print (CheckListCapa)

        # Processamento dos Erros
        erros = []
        for pagina_num, pagina in enumerate(extract_pages(caminho_arquivo), start=1):
            erros += show_ltitem_hierarchy(pagina, page_num=pagina_num)

        if erros:
            print(f"{paginas_status}\nForam encontrados os seguintes ERROS de: fonte/tamanho ❌:\n")
            palavraCompletaAux = []
            palavraCompleta = []
            errors = []
            for erro in erros:
                letra = erro['text']
                #print (letra)
                palavraCompletaAux.append(letra)
                if (letra == ''):
                    palavraAdicional = ''.join(palavraCompletaAux)  # Concatena todos os caracteres em uma string
                    palavraAdicional = palavraAdicional.strip()  # Remove qualquer espaço vazio ou string vazia no final

                    errors.append({
                        'page': erro['page'],
                        'palavra': palavraAdicional,
                        'fontname': erro['fontname'],
                        'size': erro['size'],
                        'error': erro['error']
                    })
                    #print (palavraAdicional)
                    palavraCompleta.append(palavraAdicional)
                    palavraCompletaAux.clear()

            for erro in errors:
                print(f"Página {erro['page']}: Palavra '{erro['palavra']}' - Fonte: {erro['fontname']} - Tamanho: {erro['size']}pt - Erro: {erro['error']}")                    
                    
        else:
            print(f"{paginas_status} Todas as fontes estão em Arial e o tamanho é 10 ou maior.")

    except Exception as e:
        return f"Erro ao tentar verificar a fonte e o tamanho: {e}"

# Exemplo de uso:
caminho_do_arquivo = 'C:/Users/danie/Downloads/EMT_6398.pdf'
resultado = verificar_pdf(caminho_do_arquivo)
print(resultado)


O documento está no formato A4 ✔️.
Check List Capa:

**Verificação de Informações**

1. **Título do Projeto**: Título do Projeto: ComuniCloud **✔️** (ComuniCloud - Ensino Médio Técnico)
2. **Categoria Disputada**: Categoria disputada: Ensino Médio Técnico **✔️** (Ensino Médio Técnico)
3. **Nome da Equipe**: Nome da Equipe: Não encontrado **❌**
4. **Nomes dos Integrantes ou Nome do Integrante**: Nomes dos integrantes: Heitor de Abreu, Leonardo Augusto de Matos Cela Borges **✔️** (Integrantes: Heitor de Abreu, Leonardo Augusto de Matos Cela Borges)
5. **Cursos que Estão Matriculados**: Cursos matriculados: Ensino Médio Técnico em Administração **✔️** (Cursando Ensino Médio Técnico em Administração, Senac Ribeirão Preto)

Let me know if you need anything else!
O arquivo é um PDF ✔️.
O Arquivo tem menos de 22 páginas ✔️. Total de páginas: 11. Todas as fontes estão em Arial e o tamanho é 10 ou maior.
None


In [249]:
caminho_arquivo = 'C:/Users/danie/Downloads/EMT_6398.pdf'
# Processamento dos Erros
erros = []
for pagina_num, pagina in enumerate(extract_pages(caminho_arquivo), start=1):
    erros += show_ltitem_hierarchy(pagina, page_num=pagina_num)
    print (pagina)

if erros:
    #print(f"{paginas_status}\nForam encontrados os seguintes ERROS de: fonte/tamanho ❌:\n")
    palavraCompletaAux = []
    palavraCompleta = []
    errors = []
    for erro in erros:
        letra = erro['text']
        #print (letra)
        palavraCompletaAux.append(letra)
        if (letra == ''):
            palavraAdicional = ''.join(palavraCompletaAux)  # Concatena todos os caracteres em uma string
            palavraAdicional = palavraAdicional.strip()  # Remove qualquer espaço vazio ou string vazia no final

            errors.append({
                'page': erro['page'],
                'palavra': palavraAdicional,
                'fontname': erro['fontname'],
                'size': erro['size'],
                'error': erro['error']
            })
            #print (palavraAdicional)
            palavraCompleta.append(palavraAdicional)
            palavraCompletaAux.clear()

    for erro in errors:
        print(f"Página {erro['page']}: Palavra '{erro['palavra']}' - Fonte: {erro['fontname']} - Tamanho: {erro['size']}pt - Erro: {erro['error']}")                    
            

<LTPage(1) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(2) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(3) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(4) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(5) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(6) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(7) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(8) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(9) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(10) 0.000,0.000,596.000,842.000 rotate=0>
<LTPage(11) 0.000,0.000,596.000,842.000 rotate=0>


In [235]:
clientGroq = Groq(api_key="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")

textoExtraido = '''
O documento está no formato A4 ✔️.
Texto extraído da primeira página:

E T I Q U E T A S   D O   F U T U R O  

PLANO
EMPREENDEDOR

Cursos Técnicos

Loopar

Thaina Lourenço

Daniela Almeida

Stefhany Fortunato 

Natalia Soares

Senac São Paulo, 2023

Curso Técnico em Estilismo e Coordenação de Moda
'''

prompt = f"""
Com base no texto fornecido, identifique se estão presentes as seguintes informações:

1. Título do Projeto: Identifique se há um título que possa ser reconhecido como o nome do projeto.
2. Categoria Disputada: Determine se há menção a uma categoria como curso técnico, graduação, pós-graduação, ou outras especificadas posteriormente.
3. Nome da Equipe: Verifique se há um nome que identifica uma equipe.
4. Nomes dos Integrantes ou Nome do Integrante: Identifique se são listados os nomes dos integrantes ou de um integrante específico.
5. Cursos que Estão Matriculados: Verifique se há menção aos cursos nos quais os integrantes estão matriculados.

Instruções adicionais para o modelo:

- Se um campo não estiver claramente identificado, informe que o campo não foi encontrado.
- Responda em um formato estruturado, listando cada item como um tópico (checklist) seguido de “Encontrado” ou “Não Encontrado”. Inclua o trecho do texto correspondente, se possível.
- Utilize ícones de check (✔️) e uncheck (❌) para indicar a presença ou ausência de cada informação.

Texto extraído:
{textoExtraido}
"""

chat_completion = clientGroq.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)


response = chat_completion.choices[0].message.content
print(response)

ChatCompletion(choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChoiceMessage(content='Aqui está a resposta estruturada:\n\n**1. Título do Projeto:**\n✔️ Encontrado: "E T I Q U E T A S   D O   F U T U R O "\n"Título do Projeto: PLANO EMPREENDEDOR" - Embora não esteja explicitamente chamado de "título do projeto", o texto menciona "PLANO EMPREENDEDOR", que pode ser interpretado como o título do projeto.\n\n**2. Categoria Disputada:**\n✔️ Encontrado: "Cursos Técnicos" e "Curso Técnico em Estilismo e Coordenação de Moda" - O texto menciona "Cursos Técnicos" e um curso específico, "Curso Técnico em Estilismo e Coordenação de Moda", que é um nível de formação técnico.\n\n**3. Nome da Equipe:**\n✔️ Encontrado: "Loopar" - O texto menciona a equipe "Loopar".\n\n**4. Nomes dos Integrantes ou Nome do Integrante:**\n✔️ Encontrado: Nomes dos integrantes: Thaina Lourenço, Daniela Almeida, Stefhany Fortunato, Natalia Soares - O texto lista os nomes dos integrantes da equipe.\n\n

In [236]:
response = chat_completion.choices[0].message.content
print(response)

Aqui está a resposta estruturada:

**1. Título do Projeto:**
✔️ Encontrado: "E T I Q U E T A S   D O   F U T U R O "
"Título do Projeto: PLANO EMPREENDEDOR" - Embora não esteja explicitamente chamado de "título do projeto", o texto menciona "PLANO EMPREENDEDOR", que pode ser interpretado como o título do projeto.

**2. Categoria Disputada:**
✔️ Encontrado: "Cursos Técnicos" e "Curso Técnico em Estilismo e Coordenação de Moda" - O texto menciona "Cursos Técnicos" e um curso específico, "Curso Técnico em Estilismo e Coordenação de Moda", que é um nível de formação técnico.

**3. Nome da Equipe:**
✔️ Encontrado: "Loopar" - O texto menciona a equipe "Loopar".

**4. Nomes dos Integrantes ou Nome do Integrante:**
✔️ Encontrado: Nomes dos integrantes: Thaina Lourenço, Daniela Almeida, Stefhany Fortunato, Natalia Soares - O texto lista os nomes dos integrantes da equipe.

**5. Cursos que Estão Matriculados:**
✔️ Encontrado: "Curso Técnico em Estilismo e Coordenação de Moda" - O texto menciona 

In [ ]:
#caminho_do_arquivo = 'C:/Users/danie/Downloads/EMT_6398.pdf'
#caminho_do_arquivo = 'C:/Users/danie/Downloads/Prog_8938.pdf'
#caminho_do_arquivo = 'C:/Users/danie/Downloads/Grad_8798.pdf'
#caminho_do_arquivo = 'C:/Users/danie/Downloads/Tec_5283_LINDO.pdf'

In [123]:


def show_ltitem_hierarchy(o: Any, depth=0, page_num=0, errors=None):
    """Show location and text of LTItem and all its descendants"""
    if errors is None:
        errors = []

    '''
    if depth == 0:
        print('element                        fontname             text')
        print('------------------------------ -------------------- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '
        f'{get_optional_fontinfo(o):<20.20s} '
        f'{get_optional_text(o)}'
    )
    '''

    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        # Verifica se a fonte é Arial e se o tamanho é pelo menos 10
        if 'Arial' not in o.fontname or o.size < 9.:
            errors.append({
                'page': page_num,
                'text': get_optional_text(o),
                'fontname': o.fontname,
                'size': o.size,
                'error': 'Fonte diferente de Arial' if 'Arial' not in o.fontname else 'Tamanho inferior a 10'
            })

    if isinstance(o, Iterable):
        for i in o:
            show_ltitem_hierarchy(i, depth=depth + 1, page_num=page_num, errors=errors)

    return errors

def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of class"""
    return '  ' * depth + o.__class__.__name__

def get_optional_fontinfo(o: Any) -> str:
    """Font info of LTChar if available, otherwise empty string"""
    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        return f'{o.fontname} {round(o.size)}pt'
    return ''

def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):
        return o.get_text().strip()
    return ''

In [192]:
from typing import Iterable, Any
from pdfminer.high_level import extract_pages
import os
from PyPDF2 import PdfReader

def show_ltitem_hierarchy(o: Any, depth=0, page_num=0, errors=None):
    """Show location and text of LTItem and all its descendants"""
    if errors is None:
        errors = []

    '''
    if depth == 0:
        print('element                        fontname             text')
        print('------------------------------ -------------------- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '
        f'{get_optional_fontinfo(o):<20.20s} '
        f'{get_optional_text(o)}'
    )
    '''

    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        # Verifica se a fonte é Arial e se o tamanho é pelo menos 10
        if 'Arial' not in o.fontname or o.size < 9.:
            errors.append({
                'page': page_num,
                'text': get_optional_text(o),
                'fontname': o.fontname,
                'size': o.size,
                'error': 'Fonte diferente de Arial' if 'Arial' not in o.fontname else 'Tamanho inferior a 10'
            })

    if isinstance(o, Iterable):
        for i in o:
            show_ltitem_hierarchy(i, depth=depth + 1, page_num=page_num, errors=errors)

    return errors

def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of class"""
    return '  ' * depth + o.__class__.__name__

def get_optional_fontinfo(o: Any) -> str:
    """Font info of LTChar if available, otherwise empty string"""
    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        return f'{o.fontname} {round(o.size)}pt'
    return ''

def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):
        return o.get_text().strip()
    return ''

def verificar_pdf(caminho_arquivo):
    # Verifica se o arquivo existe
    if not os.path.isfile(caminho_arquivo):
        return f"Arquivo '{caminho_arquivo}' não encontrado."

    # Verifica se o arquivo é um PDF
    if not caminho_arquivo.lower().endswith('.pdf'):
        return "O arquivo não é um PDF."

    try:
        # Abre e lê o PDF
        with open(caminho_arquivo, 'rb') as arquivo_pdf:
            leitor_pdf = PdfReader(arquivo_pdf)
            numero_paginas = len(leitor_pdf.pages)
            
            if numero_paginas < 22:
                paginas_status = f"O arquivo é um PDF e tem menos de 22 páginas. Total de páginas: {numero_paginas}."
            else:
                paginas_status = f"O arquivo é um PDF e tem 22 páginas ou mais. Total de páginas: {numero_paginas}."
    
    except Exception as e:
        return f"Erro ao tentar ler o PDF: {e}"

    try:
        erros = []
        for pagina_num, pagina in enumerate(extract_pages(caminho_arquivo), start=1):
            erros += show_ltitem_hierarchy(pagina, page_num=pagina_num)

        if erros:
            print(f"{paginas_status} Foram encontrados os seguintes erros de fonte/tamanho:\n")
            palavraCompletaAux = []
            palavraCompleta = []
            errors = []
            for erro in erros:
                letra = erro['text']
                #print (letra)
                palavraCompletaAux.append(letra)
                if (letra == ''):
                    palavraAdicional = ''.join(palavraCompletaAux)  # Concatena todos os caracteres em uma string
                    palavraAdicional = palavraAdicional.strip()  # Remove qualquer espaço vazio ou string vazia no final

                    errors.append({
                        'page': erro['page'],
                        'palavra': palavraAdicional,
                        'fontname': erro['fontname'],
                        'size': erro['size'],
                        'error': erro['error']
                    })
                    #print (palavraAdicional)
                    palavraCompleta.append(palavraAdicional)
                    palavraCompletaAux.clear()

            for erro in errors:
                print(f"Página {erro['page']}: Palavra '{erro['palavra']}' - Fonte: {erro['fontname']} - Tamanho: {erro['size']}pt - Erro: {erro['error']}")                    
                    
        else:
            print(f"{paginas_status} Todas as fontes estão em Arial e o tamanho é 10 ou maior.")

    except Exception as e:
        return f"Erro ao tentar verificar a fonte e o tamanho: {e}"

# Exemplo de uso:
caminho_do_arquivo = 'C:/Users/danie/Downloads/Tec_5283_LINDO.pdf'
resultado = verificar_pdf(caminho_do_arquivo)
print(resultado)


O arquivo é um PDF e tem menos de 22 páginas. Total de páginas: 19. Foram encontrados os seguintes erros de fonte/tamanho:

Página 3: Palavra 'Equipamentos' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'de' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'modelagem' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'ecosturaCelularComputadorInternet' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'Wi-FiSoftwares' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'de' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'edição' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 5: Palavra 'vetorial5cm' - Fonte: AAAAAA+ArialMT 

In [171]:
erros

[{'page': 3,
  'text': 'E',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'q',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'u',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'i',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'p',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'a',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'm',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior a 10'},
 {'page': 3,
  'text': 'e',
  'fontname': 'AAAAAA+ArialMT',
  'size': 6.226069276580432,
  'error': 'Tamanho inferior 

In [191]:
from typing import Iterable, Any
from pdfminer.high_level import extract_pages
import os
from PyPDF2 import PdfReader

def show_ltitem_hierarchy(o: Any, depth=0, page_num=0, errors=None):
    """Show location and text of LTItem and all its descendants"""
    if errors is None:
        errors = []

    '''
    if depth == 0:
        print('element                        fontname             text')
        print('------------------------------ -------------------- -----')

    print(
        f'{get_indented_name(o, depth):<30.30s} '
        f'{get_optional_fontinfo(o):<20.20s} '
        f'{get_optional_text(o)}'
    )
    '''

    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        # Verifica se a fonte é Arial e se o tamanho é pelo menos 10
        if 'Arial' not in o.fontname or o.size < 9.:
            errors.append({
                'page': page_num,
                'text': get_optional_text(o),
                'fontname': o.fontname,
                'size': o.size,
                'error': 'Fonte diferente de Arial' if 'Arial' not in o.fontname else 'Tamanho inferior a 10'
            })

    if isinstance(o, Iterable):
        for i in o:
            show_ltitem_hierarchy(i, depth=depth + 1, page_num=page_num, errors=errors)

    return errors

def get_indented_name(o: Any, depth: int) -> str:
    """Indented name of class"""
    return '  ' * depth + o.__class__.__name__

def get_optional_fontinfo(o: Any) -> str:
    """Font info of LTChar if available, otherwise empty string"""
    if hasattr(o, 'fontname') and hasattr(o, 'size'):
        return f'{o.fontname} {round(o.size)}pt'
    return ''

def get_optional_text(o: Any) -> str:
    """Text of LTItem if available, otherwise empty string"""
    if hasattr(o, 'get_text'):
        return o.get_text().strip()
    return ''





# Abre e lê o PDF
with open(caminho_arquivo, 'rb') as arquivo_pdf:
    leitor_pdf = PdfReader(arquivo_pdf)
    numero_paginas = len(leitor_pdf.pages)
    
    if numero_paginas < 22:
        paginas_status = f"O arquivo é um PDF e tem menos de 22 páginas. Total de páginas: {numero_paginas}."
    else:
        paginas_status = f"O arquivo é um PDF e tem 22 páginas ou mais. Total de páginas: {numero_paginas}."


erros = []
for pagina_num, pagina in enumerate(extract_pages(caminho_arquivo), start=1):
    erros += show_ltitem_hierarchy(pagina, page_num=pagina_num)

if erros:
    print(f"{paginas_status} Foram encontrados os seguintes erros de fonte/tamanho:\n")
    palavraCompletaAux = []
    palavraCompleta = []
    errors = []
    for erro in erros:
        letra = erro['text']
        #print (letra)
        palavraCompletaAux.append(letra)
        if (letra == ''):
            palavraAdicional = ''.join(palavraCompletaAux)  # Concatena todos os caracteres em uma string
            palavraAdicional = palavraAdicional.strip()  # Remove qualquer espaço vazio ou string vazia no final

            errors.append({
                'page': erro['page'],
                'palavra': palavraAdicional,
                'fontname': erro['fontname'],
                'size': erro['size'],
                'error': erro['error']
            })
            #print (palavraAdicional)
            palavraCompleta.append(palavraAdicional)
            palavraCompletaAux.clear()    
            
else:
    print(f"{paginas_status} Todas as fontes estão em Arial e o tamanho é 10 ou maior.")


# Exemplo de uso:
caminho_arquivo = 'C:/Users/danie/Downloads/Tec_5283_LINDO.pdf'
resultado = verificar_pdf(caminho_do_arquivo)
print(resultado)


O arquivo é um PDF e tem menos de 22 páginas. Total de páginas: 19. Foram encontrados os seguintes erros de fonte/tamanho:

Página 3: Palavra 'Equipamentos' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'de' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'modelagem' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'ecosturaCelularComputadorInternet' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'Wi-FiSoftwares' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'de' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 3: Palavra 'edição' - Fonte: AAAAAA+ArialMT - Tamanho: 6.226069276580432pt - Erro: Tamanho inferior a 10
Página 5: Palavra 'vetorial5cm' - Fonte: AAAAAA+ArialMT 

In [189]:
for x in errors:
    print (x)

{'page': 3, 'palavra': 'Equipamentos', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'de', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'modelagem', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'ecosturaCelularComputadorInternet', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'Wi-FiSoftwares', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'de', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 3, 'palavra': 'edição', 'fontname': 'AAAAAA+ArialMT', 'size': 6.226069276580432, 'error': 'Tamanho inferior a 10'}
{'page': 5, 'palavra': 'vetorial5cm', 'fontname': 'AAAAAA+ArialMT', 'size': 2.775293266703329, '